# Prediction May and June data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import warnings
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Data loading and engineering

In [37]:
data_0 = pd.read_csv('cluster_0.csv', index_col=0)
data_1 = pd.read_csv('cluster_1.csv', index_col=0)
data_2 = pd.read_csv('cluster_2.csv', index_col=0)
data_3 = pd.read_csv('cluster_3.csv', index_col=0)
data_4 = pd.read_csv('cluster_4.csv', index_col=0)
data_0 = data_0.set_index(pd.date_range('01/01/2016', periods=int(data_0.shape[0]), freq='H'))
data_0.drop(['value_mean', 'value_std'], axis=1, inplace=True)
data_0.columns = [str(int(i)+1) for i in list(data_0.columns)]
data_1 = data_1.set_index(pd.date_range('01/01/2016', periods=int(data_1.shape[0]), freq='H'))
data_1.drop(['value_mean', 'value_std'], axis=1, inplace=True)
data_1.columns = [str(int(i)+1) for i in list(data_1.columns)]
data_2 = data_2.set_index(pd.date_range('01/01/2016', periods=int(data_2.shape[0]), freq='H'))
data_2.drop(['value_mean', 'value_std'], axis=1, inplace=True)
data_2.columns = [str(int(i)+1) for i in list(data_2.columns)]
data_3 = data_3.set_index(pd.date_range('01/01/2016', periods=int(data_3.shape[0]), freq='H'))
data_3.drop(['value_mean', 'value_std'], axis=1, inplace=True)
data_3.columns = [str(int(i)+1) for i in list(data_3.columns)]
data_4 = data_4.set_index(pd.date_range('01/01/2016', periods=int(data_4.shape[0]), freq='H'))
data_4.drop(['value_mean', 'value_std'], axis=1, inplace=True)
data_4.columns = [str(int(i)+1) for i in list(data_4.columns)]

In [38]:
def make_regressive_features(series):
    feature_df = pd.DataFrame(series).drop(pd.DataFrame(series).columns, axis=1) # df to store regressive features for every series
    for i in range(1, 6):
        feature_df['sin_value_{}'.format(i)] = np.sin(np.arange(series.shape[0]) * 2*np.pi * i/720)
        feature_df['cos_value_{}'.format(i)] = np.cos(np.arange(series.shape[0]) * 2*np.pi * i/720)
    feature_df['roll_12'.format(i)] = series.rolling(12).mean()
    feature_df['roll_24'.format(i)] = series.rolling(24).mean()
    feature_df['roll_96'.format(i)] = series.rolling(96).mean()
    feature_df['roll_168'.format(i)] = series.rolling(168).mean()
    isWeekend = lambda x: 1 if x in (5, 6) else 0
    feature_df['is_weekend'] = [isWeekend(i) for i in feature_df.index.dayofweek]
    feature_df['is_new_year'] = [(lambda x: 1 if (x[0]==1 & x[1]==1) else 0)(x) for x in zip(feature_df.index.day,
                                                                                           feature_df.index.month)]
    feature_df['is_martin_luther'] = [(lambda x: 1 if (x[0]==18 & x[1]==1) else 0)(x) for x in zip(feature_df.index.day,
                                                                                           feature_df.index.month)]
    feature_df['is_presidents_day'] = [(lambda x: 1 if (x[0]==15 & x[1]==2) else 0)(x) for x in zip(feature_df.index.day,
                                                                                           feature_df.index.month)]
    feature_df['is_emancipation_day'] = [(lambda x: 1 if (x[0]==15 & x[1]==4) else 0)(x) for x in zip(feature_df.index.day,
                                                                                           feature_df.index.month)]
    return feature_df

In [39]:
cluster_region_dict = {
    'cluster 1': (1076, 1126, 1127, 1128, 1129, 1132, 1180, 1181, 1230, 1231, 1232, 1233, 1234, 1282,
 1283, 1339, 1383, 1431, 1684, 1734, 2069, 2119),
    'cluster 2': (1075, 1125, 1221, 1227, 1272, 1326, 1331, 1382, 1434, 1441, 1480, 1482, 1483, 1530,
 1532, 1533, 1580, 1630, 1733, 1783, 2068, 2118, 2168),
    'cluster 3': (1173, 1174, 1175, 1176, 1183, 1225, 1278, 1388, 1389, 1390, 1436, 1437, 1438, 1439,
 1442),
    'cluster 4': (1130, 1131, 1172, 1177, 1178, 1179, 1222, 1223, 1224, 1228, 1229, 1273, 1274, 1279,
 1327, 1376, 1377, 1378, 1380, 1426),
    'cluster 5': (1077, 1182, 1184, 1235, 1280, 1281, 1284, 1285, 1286, 1287, 1332, 1333, 1334, 1335,
 1336, 1337, 1338, 1384, 1385, 1386, 1387, 1435)
}
best_params_dict = {
    'cluster 1': (2, 7, 2, 1),
    'cluster 2': (2, 2, 2, 2),
    'cluster 3': (2, 2, 2, 0),
    'cluster 4': (2, 2, 2, 2),
    'cluster 5': (1, 2, 2, 2)
}

In [40]:
def normalize_data_may(trips_by_regions_may, cluster):
    trips_by_regions_may = np.transpose(trips_by_regions_may[trips_by_regions_may['region'].isin(cluster_region_dict[cluster])].drop(['west', 'east', 'south', 'north', 'region'], axis=1))
    trips_by_regions_may.columns = [i+1 for i in list(trips_by_regions_may.columns)]
    trips_by_regions_may = trips_by_regions_may.set_index(pd.date_range('05/01/2016', periods=int(trips_by_regions_may.shape[0]), freq='H'))
    return trips_by_regions_may

In [41]:
trips_by_regions_may = pd.read_csv('trips_by_regions_may.csv')

trips_by_regions_may_cluster_0 = normalize_data_may(trips_by_regions_may, 'cluster 1')
trips_by_regions_may_cluster_1 = normalize_data_may(trips_by_regions_may, 'cluster 2')
trips_by_regions_may_cluster_2 = normalize_data_may(trips_by_regions_may, 'cluster 3')
trips_by_regions_may_cluster_3 = normalize_data_may(trips_by_regions_may, 'cluster 4')
trips_by_regions_may_cluster_4 = normalize_data_may(trips_by_regions_may, 'cluster 5')


In [42]:
data_0.columns = [int(i) for i in data_0.columns]
data_1.columns = [int(i) for i in data_1.columns]
data_2.columns = [int(i) for i in data_2.columns]
data_3.columns = [int(i) for i in data_3.columns]
data_4.columns = [int(i) for i in data_4.columns]

In [43]:
data_0 = pd.concat([data_0, trips_by_regions_may_cluster_0])
data_1 = pd.concat([data_1, trips_by_regions_may_cluster_1])
data_2 = pd.concat([data_2, trips_by_regions_may_cluster_2])
data_3 = pd.concat([data_3, trips_by_regions_may_cluster_3])
data_4 = pd.concat([data_4, trips_by_regions_may_cluster_4])

In [44]:
data_0.head()

,1076,1126,1127,1128,1129,1132,1180,1181,1230,1231,...,1234,1282,1283,1339,1383,1431,1684,1734,2069,2119
2016-01-01 00:00:00,80.0,77.0,319.0,402.0,531.0,267.0,759.0,930.0,1001.0,1273.0,...,664.0,768.0,1076.0,84.0,22.0,5.0,2.0,2.0,41.0,70.0
2016-01-01 01:00:00,91.0,134.0,404.0,420.0,370.0,224.0,518.0,671.0,1116.0,1514.0,...,567.0,1062.0,1178.0,81.0,15.0,5.0,1.0,5.0,4.0,47.0
2016-01-01 02:00:00,90.0,110.0,393.0,425.0,313.0,138.0,401.0,503.0,962.0,1363.0,...,705.0,1060.0,1053.0,63.0,10.0,4.0,0.0,3.0,0.0,69.0
2016-01-01 03:00:00,32.0,62.0,252.0,399.0,324.0,166.0,391.0,523.0,968.0,1202.0,...,615.0,614.0,610.0,44.0,5.0,5.0,0.0,2.0,1.0,21.0
2016-01-01 04:00:00,24.0,53.0,145.0,254.0,264.0,145.0,388.0,381.0,646.0,647.0,...,379.0,319.0,401.0,18.0,4.0,5.0,1.0,0.0,0.0,26.0


In [45]:
def count_q(real, predict):
    q = []
    for i,j in zip(real, predict):
        q.append(abs(i - j))
    return np.mean(q)

In [46]:
Q_final = []
clusters = {'cluster 1': data_0, 
            'cluster 2': data_1,
            'cluster 3': data_2,
            'cluster 4': data_3,
            'cluster 5': data_4}

# May predictions

In [ ]:
%%time
time_range = [i for i in list(pd.date_range('05/01/2016', periods=int(24*31), freq='H'))]
d, D = 0, 1
for cluster_name, data_cluster in clusters.items():
    param = best_params_dict[cluster_name]
    regressor = LinearRegression()
    Qs = []
    for series in data_cluster.columns:
        features = make_regressive_features(data_cluster[series])
        regressor.fit(features.fillna(0), data_cluster[series])
        data = pd.DataFrame(data_cluster[series])
        data['reg_predictions'] = regressor.predict(features.fillna(0))
        data['residuals'] = data[series] - data['reg_predictions']
        data['pred'] = np.nan
        try:
            model_res = sm.tsa.statespace.SARIMAX(data['residuals'][:time_range[0]], 
                                                 order=(param[0], d, param[1]), 
                                                 seasonal_order=(param[2], D, param[3], 24)).fit(disp=-1)
            model_fitted = sm.tsa.statespace.SARIMAX(data['residuals'],order=(param[0], d, param[1]), 
                                                 seasonal_order=(param[2], D, param[3], 24)).filter(model_res.params)
        except:
            continue
        Q = []
        for time in time_range[:-6]:
            predicted_residuals = model_fitted.predict(time, time+6, dynamic=True)
            data['pred'][time: time+6] = predicted_residuals + data['reg_predictions'][time: time+6]
            Q.append(count_q(data['pred'][time: time+6], data[series][time: time+6]))
#             if time.hour == 0:
#                 print(time.day)
        Qs.append(np.mean(Q))
        print('{} series finihshed ({})'.format(series, cluster_name))
    Q_final.append(np.mean(Qs))


In [35]:
print('Q = {}'.format(np.mean(Q_final)))

Q = 20.65831844879667


# June predictions

In [47]:
def normalize_data_june(trips_by_regions_june, cluster):
    trips_by_regions_june = np.transpose(trips_by_regions_june[trips_by_regions_june['region'].isin(cluster_region_dict[cluster])].drop(['west', 'east', 'south', 'north', 'region'], axis=1))
    trips_by_regions_june.columns = [i+1 for i in list(trips_by_regions_june.columns)]
    trips_by_regions_june = trips_by_regions_june.set_index(pd.date_range('06/01/2016', periods=int(trips_by_regions_june.shape[0]), freq='H'))
    return trips_by_regions_june

In [48]:
trips_by_regions_june = pd.read_csv('trips_by_regions_june.csv')

trips_by_regions_june_cluster_0 = normalize_data_june(trips_by_regions_june, 'cluster 1')
trips_by_regions_june_cluster_1 = normalize_data_june(trips_by_regions_june, 'cluster 2')
trips_by_regions_june_cluster_2 = normalize_data_june(trips_by_regions_june, 'cluster 3')
trips_by_regions_june_cluster_3 = normalize_data_june(trips_by_regions_june, 'cluster 4')
trips_by_regions_june_cluster_4 = normalize_data_june(trips_by_regions_june, 'cluster 5')

In [49]:
data_0_final = pd.concat([data_0, trips_by_regions_june_cluster_0])
data_1_final = pd.concat([data_1, trips_by_regions_june_cluster_1])
data_2_final = pd.concat([data_2, trips_by_regions_june_cluster_2])
data_3_final = pd.concat([data_3, trips_by_regions_june_cluster_3])
data_4_final = pd.concat([data_4, trips_by_regions_june_cluster_4])

In [50]:
clusters = {'cluster 1': data_0_final, 
            'cluster 2': data_1_final,
            'cluster 3': data_2_final,
            'cluster 4': data_3_final,
            'cluster 5': data_4_final}

In [ ]:
%%time
time_range = [i for i in list(pd.date_range('05/31/2016 23:00:00', periods=int(24*30), freq='H'))]
result = pd.DataFrame(columns=('id', 'y'))
a=0
d, D = 0, 1
for cluster_name, data_cluster in clusters.items():
    param = best_params_dict[cluster_name]
    regressor = LinearRegression()
    for series in data_cluster.columns:
        features = make_regressive_features(data_cluster[series])
        regressor.fit(features.fillna(0), data_cluster[series])
        data = pd.DataFrame(data_cluster[series])
        data['reg_predictions'] = regressor.predict(features.fillna(0))
        data['residuals'] = data[series] - data['reg_predictions']
        data['pred'] = np.nan
        try:
            model_res = sm.tsa.statespace.SARIMAX(data['residuals'][:time_range[0]], 
                                                 order=(param[0], d, param[1]), 
                                                 seasonal_order=(param[2], D, param[3], 24)).fit(disp=-1)
            model_fitted = sm.tsa.statespace.SARIMAX(data['residuals'],order=(param[0], d, param[1]), 
                                                 seasonal_order=(param[2], D, param[3], 24)).filter(model_res.params)
        except:
            pass
        for time in time_range[:-5]:
            predicted_residuals = model_fitted.predict(time+1, time+6, dynamic=True)
            data['pred'][time+1: time+6] = predicted_residuals + data['reg_predictions'][time+1: time+6]
            for i in range(6):
                result = result.append([{'id' : '{}_{}-{}-{}_{}_{}'.format(series, time.year, time.month, time.day, time.hour, i+1),
                               'y':data['pred'][time + i +1]}])
        print('{} done'.format(series))
    print('{} done'.format(cluster_name))
                

### Changing to a valid format form

In [9]:
new_ids = []
for i in result['id'].values:
    if i[10]=='5':
        new_ids.append(i.replace('-5-', '-05-'))
    else:
        new_ids.append(i.replace('-6-', '-06-'))
result['id'] = new_ids

In [10]:
new_ids = []
v = ['-1_', '-2_', '-3_', '-4_', '-5_', '-6_', '-7_', '-8_', '-9_']
v_ = ['-01_', '-02_', '-03_', '-04_', '-05_', '-06_', '-07_', '-08_', '-09_']
for i in result['id'].values:
    target = i[12:15]
    if target in v:
        new_ids.append(i.replace(target, v_[v.index(target)]))
    else:
        new_ids.append(i)
result['id'] = new_ids

In [15]:
result.head()

,id,y
0,1076_2016-05-31_23_1,48.155182
1,1076_2016-05-31_23_2,28.972377
2,1076_2016-05-31_23_3,20.879477
3,1076_2016-05-31_23_4,16.067401
4,1076_2016-05-31_23_5,14.525768


In [14]:
result.shape

(437580, 2)

In [13]:
result.to_csv('result_arima_final.csv', index=False)

kaggle link: https://www.kaggle.com/c/yellowtaxi/leaderboard (87th place, 31.11283 (Svyatoslav Oreshin)